# **Preparation of PIMUS Network for GTAModel**

### **General pipeline overview**

![methology](imgs/pipeline.jpg)

### **Pipeline**

* **Install modules**

In [ ]:
!pip install \
    --extra-index-url=https://pypi.nvidia.com \
    cudf-cu11 dask-cudf-cu11 cuml-cu11 cugraph-cu11 cuspatial-cu11 cuproj-cu11 cuxfilter-cu11 cucim



**Convenience of use RAPIDS**


![speedup](https://developer-blogs.nvidia.com/wp-content/uploads/2023/03/performance-comparison-pandas-cudf-1-625x386.png)

In [ ]:
!nvidia-smi

* **Import modules**

In [1]:
import cudf
import cuspatial
import cupy
import geopandas
from glob import glob
import pandas as pd
import numpy as np
from shapely.geometry import *
from shapely import wkt
import zipfile
#local import
from Converters import Converters 

* **Convert Net file to csv's**

In [2]:
#unzip the Network file
!unzip NetModel/BaseVisumProject/network.zip
#create dir to export csv
!mkdir Base_Network
conv = Converters()
#convert .net to csv in the created dir
conv.net2csv('network.net','Base_Network/')

Archive:  NetModel/BaseVisumProject/network.zip
  inflating: network.net             
Parsing table:  Versionblock
Use delimiter ";"
Exported 1 objects in table: Versionblock - saved to Base_Network/Versionblock.csv
Parsing table:  User-definedattributes
Exported 151 objects in table: User-definedattributes - saved to Base_Network/User-definedattributes.csv
Parsing table:  Time-varyingattributes
Exported 1 objects in table: Time-varyingattributes - saved to Base_Network/Time-varyingattributes.csv
Parsing table:  Calendarperiods
Exported 1 objects in table: Calendarperiods - saved to Base_Network/Calendarperiods.csv
Parsing table:  Validdays
Exported 1 objects in table: Validdays - saved to Base_Network/Validdays.csv
Parsing table:  Network
Exported 1 objects in table: Network - saved to Base_Network/Network.csv
Parsing table:  Transportsystems
Exported 19 objects in table: Transportsystems - saved to Base_Network/Transportsystems.csv
Parsing table:  Modes
Exported 7 objects in table: M

* **Extract shapefiles from each object of the network**

* **Load all objects as dataframes**

In [3]:
objDFs = {(f.split('/')[-1]).split('.')[0]: pd.read_csv(f) for f in glob('Base_Network/*.csv')}
list(objDFs.keys())

/tmp/ipykernel_8781/3508830712.py:1: DtypeWarning: Columns (242,266,267) have mixed types. Specify dtype option on import or set low_memory=False.
  objDFs = {(f.split('/')[-1]).split('.')[0]: pd.read_csv(f) for f in glob('Base_Network/*.csv')}


['Transferwalktimesbetweenstopareas',
 'Faceitems',
 'Modes',
 'Lineroutes',
 'Operators',
 'Edges',
 'Transferfares',
 'Blockitemtypes',
 'Timeprofiles',
 'Validdays',
 'Turns',
 'Mainzones',
 'Linktypes',
 'Zones',
 'Calendarperiods',
 'Surfaces',
 'Linerouteitems',
 'User-definedattributes',
 'Timeprofileitems',
 'Stoppoints',
 'Blockversions',
 'Stops',
 'Connectors',
 'Faremodel',
 'Stopareas',
 'Tickettypes',
 'Faces',
 'PuTdirections',
 'Intermediatepoints',
 'Faresystems',
 'Nodes',
 'Lines',
 'Links',
 'Network',
 'Demandsegments',
 'Time-varyingattributes',
 'Aliases',
 'Transportsystems',
 'Surfaceitems',
 'Vehiclecombinationitems',
 'Vehiclejourneys',
 'Points',
 'Versionblock',
 'FaresystemtickettypesbyDSeg',
 'Faresupplements',
 'Vehiclejourneysections',
 'Vehiclecombinations',
 'Linkpolygons',
 'Vehicleunits']

In [4]:
!nvidia-smi

Wed Sep  6 20:46:40 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.10              Driver Version: 535.86.10    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1660 ...    On  | 00000000:01:00.0  On |                  N/A |
| N/A   61C    P0              22W /  60W |   2029MiB /  6144MiB |      6%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

* **Filter all the objects with TSys proposals information**

In [4]:
# extract objects with the TSys proposals
props = ['tren_aero', 'regional', 'tren', 'corredor','alimentadora corredor']
# get the files to modify where the TSys appear
p = !grep -lR "Corredor"
# remove non csv files and get the key name
kPropFiles = [(v.split('/')[-1]).split('.')[0] for v in p if v.endswith(".csv")]
print(f'[INFO] Objects to clean: {kPropFiles}')

[INFO] Objects to clean: ['Faresystems', 'Transportsystems']


In [5]:
# extract codes of the proposals TransportSystems 
objWcode = [k for k in kPropFiles if 'CODE' in objDFs[k].columns.tolist()]
cProps = [list((objDFs[o][objDFs[o]['NAME'].str.lower() == p].CODE).to_dict().values()) for p in props for o in objWcode]
cProps = [c[0] for c in cProps if len(c) > 0] 
print(cProps)
# extract no. of the proposals TransportSystems 
objWno = list(set(kPropFiles) - set(objWcode))
nProps = [list((objDFs[o][objDFs[o]['NAME'].str.lower() == p].NO).to_dict().values()) for p in props for o in objWno]
nProps = [n[0] for n in nProps if len(n) > 0] 
print(nProps)

['RE', 'S', 'C', 'AC']
[13, 12, 11, 10, 9]


In [6]:
def cleanByList(df,column: str, values: list):
    for v in values:
        df = df[df[column].str.lower() != v]
    return df
    
for k in kPropFiles:
#cleaning objects by TSys proposals names
    objDFs[k] = cleanByList(objDFs[k],'NAME', props)
objDFs[kPropFiles[0]]

,NO,NAME,RANK,JOINTFARECOMPUTATION,FAREWEIGHT,INITIALFARE,TSYSSETNONPUTLINE\n
0,1,Alimentadora,1,0,1.0,12.0,NaN
1,2,Metro,1,0,1.0,4.5,NaN
2,3,Remanente,1,0,1.0,12.0,NaN
3,4,BRT,1,0,1.0,14.6,NaN
4,5,TM,1,0,1.0,4.5,NaN
5,6,MBUS,1,0,1.0,12.0,NaN
6,7,Remanente 15,1,0,1.0,15.0,NaN
7,8,Remanente 17,1,0,1.0,17.0,NaN


In [7]:
def cleanBycode(s:str) -> str:
    if isinstance(s,str):
        if len(s.split(',')) >0:
            return ','.join(set (s.split(',')) - set (cProps))
    return s
def dummy(r):
    return r
#cleaning objects by TSys proposals codes and removing nulls
objWtsset = [k for k in objDFs.keys() if 'TSYSSET' in objDFs[k].columns.tolist()]
print(f'[INFO] Objects to clean: {objWtsset}')
print(objDFs[objWtsset[0]])
for o in objWtsset:
    objDFs[o] = objDFs[o][~objDFs[o]['TSYSSET'].isnull()]
    objDFs[o]['TSYSSET'] = objDFs[o]['TSYSSET'].apply(cleanBycode)
objDFs[objWtsset[0]] 

[INFO] Objects to clean: ['Modes', 'Turns', 'Linktypes', 'Stoppoints', 'Connectors', 'Links', 'Vehicleunits']
     CODE                        NAME                              TSYSSET  \
0  Auto_C                  Auto Cuota                               Auto_C   
1   Ca1_C     Ca1 Camion articulado 1                                Ca1_C   
2   Ca2_C     Ca2 Camion articulado 2                                Ca2_C   
3   Ca_BD  Camion articulado Big Data                                Ca_BD   
4   Cu_BD    Camion unitario Big Data                                Cu_BD   
5    Cu_C          Cu Camion unitario                                 Cu_C   
6     Pub                     Publico  AC,Al,C,Ex,M,MBUS,R,RAE,RE,S,T,TM,W   

   INTERCHANGEABLE\n  
0                  1  
1                  1  
2                  1  
3                  1  
4                  1  
5                  1  
6                  1  


,CODE,NAME,TSYSSET,INTERCHANGEABLE\n
0,Auto_C,Auto Cuota,Auto_C,1
1,Ca1_C,Ca1 Camion articulado 1,Ca1_C,1
2,Ca2_C,Ca2 Camion articulado 2,Ca2_C,1
3,Ca_BD,Camion articulado Big Data,Ca_BD,1
4,Cu_BD,Camion unitario Big Data,Cu_BD,1
5,Cu_C,Cu Camion unitario,Cu_C,1
6,Pub,Publico,"T,Ex,RAE,W,TM,M,Al,MBUS,R",1


In [8]:
#cleaning proposals from objects by faresystemno
objWfsno = {k: [c for c in objDFs[k].columns.tolist() if 'FARESYSTEMNO' in c] for k in objDFs.keys() if len([c for c in objDFs[k].columns.tolist() if 'FARESYSTEMNO' in c]) > 0}
print(f'[INFO] Objects to clean: {objWfsno}')
for o in objWfsno.keys():
    for c in objWfsno[o]:
        objDFs[o] = objDFs[o][~objDFs[o][c].isnull()]
        objDFs[o] = objDFs[o][~objDFs[o][c].isin(nProps)]
    print(objDFs[o].head())

[INFO] Objects to clean: {'Transferfares': ['FROMFARESYSTEMNO', 'TOFARESYSTEMNO'], 'FaresystemtickettypesbyDSeg': ['FARESYSTEMNO\n']}
   FROMFARESYSTEMNO  TOFARESYSTEMNO  FARE\n
0                 1               1    12.0
1                 1               2     4.5
2                 1               3    12.0
3                 1               4    14.6
4                 1               5    12.0
   TICKETTYPENO DSEGCODE  FARESYSTEMNO\n
0             1       P1               1
1             1       P2               1
2             1       P3               1
3             1       P1               2
4             1       P2               2


In [9]:
#cleaning proposals from objects by faresystemcode

objWfscd = {k: [c for c in objDFs[k].columns.tolist() if 'SYSCODE' in c] for k in objDFs.keys() if len([c for c in objDFs[k].columns.tolist() if 'SYSCODE' in c]) > 0}
print(f'[INFO] Objects to clean: {objWfscd}')
for o in objWfscd.keys():
    for c in objWfscd[o]:
        objDFs[o] = objDFs[o][~objDFs[o][c].isnull()]
        objDFs[o] = objDFs[o][~objDFs[o][c].isin(cProps)]
    print(objDFs[o].head())

[INFO] Objects to clean: {'Transferwalktimesbetweenstopareas': ['TSYSCODE'], 'Lines': ['TSYSCODE'], 'Faresupplements': ['TSYSCODE']}
   FROMSTOPAREANO  TOSTOPAREANO TSYSCODE TIME\n
0            2329          2329        W     0s
1            2330          2330        W     0s
2            2336          2336        W     0s
3            2354          2354        W     0s
4            2363          2363        W     0s
    NAME TSYSCODE  VEHCOMBNO  FARESYSTEMSET  OPERATORNO  MAINLINENAME  \
0    R_1        R          1              3           1           NaN   
1   R_10        R          1              3           1           NaN   
2  R_100        R          1              3           1           NaN   
3  R_101        R          1              3           1           NaN   
4  R_102        R          1              3           1           NaN   

   ADDVAL1  ADDVAL2  ADDVAL3\n  
0        0        0          0  
1        0        0          0  
2        0        0          0  
3       

* **Read and filter links for walk only not associated with stops** 

In [10]:
objDFs['Links']

,NO,FROMNODENO,TONODENO,NAME,TYPENO,TSYSSET,USERDIRECTION,LENGTH,NUMLANES,PLANNO,...,TYPE_LINK_CGA,TYPE_LINK_PRV,VEL_CGA,VEL_FLUJO,VEL_PRV,VEL_PUB,VIALIDAD_PROY,VOL_CAP,VOL_TESC,VOL_TPER\n
0,1,107820,107948,NaN,0,W,0,0.503km,0,0,...,99,0,0.0,0,NaN,NaN,NaN,0,0.0,0.0
1,1,107948,107820,NaN,91,"Ca2_C,Ca1_C,W,Cu_C,Ca_BD,Cu_BD,Auto_C,R",1,0.503km,2,0,...,99,91,22.0,40,30.0,26.795,NaN,1,0.0,0.0
2,2,2327,2328,NaN,0,W,0,0.047km,0,0,...,99,0,0.0,0,NaN,NaN,0.0,0,0.0,0.0
3,2,2328,2327,NaN,1,W,1,0.047km,0,0,...,99,1,0.0,0,NaN,NaN,0.0,0,0.0,0.0
4,3,2329,2330,NaN,0,W,0,0.263km,0,0,...,99,0,0.0,0,NaN,NaN,0.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376313,201549,140382,131119,NaN,0,"Auto_C,W,R",1,0.044km,1,0,...,99,0,0.0,30,NaN,0.000,NaN,0,0.0,0.0
376316,201553,116460,140834,NaN,94,"Ca2_C,Ca1_C,Cu_C,Ca_BD,Cu_BD,Auto_C",0,11.980km,3,0,...,99,94,60.0,80,60.0,0.000,0.0,1,0.0,0.0
376317,201553,140834,116460,NaN,94,"Ca2_C,Ca1_C,Cu_C,Ca_BD,Cu_BD,Auto_C",1,11.980km,3,0,...,99,94,60.0,80,60.0,0.000,0.0,1,0.0,0.0
376318,201554,140287,140834,NaN,94,"Ca2_C,Ca1_C,Cu_C,Ca_BD,Cu_BD,Auto_C",0,6.652km,3,0,...,99,94,60.0,80,60.0,0.000,0.0,1,0.0,0.0


In [11]:
nd = list(objDFs['Stoppoints']['NO'].unique())
2330 in nd

True

In [12]:
WAB_codes = ['Auto_C','Cu_C','Cu_BD','W']	
def cleanWAB(s:str) -> str:
    if isinstance(s,str):
        if len(s.split(',')) >0:
            if len(set (s.split(',')) - set (WAB_codes)) == 0:
                return np.nan
    return s

#removing only walk,auto and unitary bus links not associated with stops
objDFs['Links']['TSYSSET'] = objDFs['Links']['TSYSSET'].apply(cleanWAB)
objDFs['Links'] = objDFs['Links'].query('TSYSSET != "NaN" | @nd in FROMNODENO | @nd in TONODENO')
#removing nulls values
objDFs['Links'] = objDFs['Links'][~objDFs['Links']['TSYSSET'].isnull()]
#displaye head
objDFs['Links']

,NO,FROMNODENO,TONODENO,NAME,TYPENO,TSYSSET,USERDIRECTION,LENGTH,NUMLANES,PLANNO,...,TYPE_LINK_CGA,TYPE_LINK_PRV,VEL_CGA,VEL_FLUJO,VEL_PRV,VEL_PUB,VIALIDAD_PROY,VOL_CAP,VOL_TESC,VOL_TPER\n
1,1,107948,107820,NaN,91,"Ca2_C,Ca1_C,W,Cu_C,Ca_BD,Cu_BD,Auto_C,R",1,0.503km,2,0,...,99,91,22.0,40,30.0,26.795,NaN,1,0.00,0.0
5,3,2330,2329,NaN,90,"Ca2_C,Ca1_C,W,Cu_C,Ca_BD,Cu_BD,Auto_C,R",1,0.263km,2,0,...,99,90,19.0,30,28.0,23.822,0.0,0,0.00,0.0
10,6,2335,2336,NaN,90,"Ca2_C,Ca1_C,W,Cu_C,Ca_BD,Al,Cu_BD,Auto_C,R",0,0.332km,1,0,...,99,90,33.0,30,33.0,22.787,NaN,0,0.00,0.0
11,6,2336,2335,NaN,90,"Ca2_C,Ca1_C,W,Cu_C,Ca_BD,Al,Cu_BD,Auto_C,R",1,0.332km,1,0,...,99,90,39.0,30,39.0,12.559,NaN,0,14.24,0.0
14,8,2339,2340,NaN,90,"Ca2_C,Ca1_C,W,Cu_C,Ca_BD,Cu_BD,Auto_C,R",0,0.169km,2,0,...,99,90,11.0,30,14.0,12.349,0.0,0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376313,201549,140382,131119,NaN,0,"Auto_C,W,R",1,0.044km,1,0,...,99,0,0.0,30,NaN,0.000,NaN,0,0.00,0.0
376316,201553,116460,140834,NaN,94,"Ca2_C,Ca1_C,Cu_C,Ca_BD,Cu_BD,Auto_C",0,11.980km,3,0,...,99,94,60.0,80,60.0,0.000,0.0,1,0.00,0.0
376317,201553,140834,116460,NaN,94,"Ca2_C,Ca1_C,Cu_C,Ca_BD,Cu_BD,Auto_C",1,11.980km,3,0,...,99,94,60.0,80,60.0,0.000,0.0,1,0.00,0.0
376318,201554,140287,140834,NaN,94,"Ca2_C,Ca1_C,Cu_C,Ca_BD,Cu_BD,Auto_C",0,6.652km,3,0,...,99,94,60.0,80,60.0,0.000,0.0,1,0.00,0.0


* **Import, clean and prune non-stop nodes without links**

In [13]:
#display nodes
print(len(objDFs['Nodes']))
objDFs['Nodes'].head()

128975


,NO,CODE,NAME,TYPENO,CONTROLTYPE,MAINNODENO,USEMETHODIMPATNODE,METHODIMPATNODE,AUTOLINKORIENTATION,XCOORD,...,ICAMETHODALLWAYSTOP,ICAMETHODROUNDABOUT,ICAMETHODSIGNALIZED,ICAMETHODTWOWAYSTOP,VISTRODOANALYSIS,SIGNALWARRANTPOPISBELOW10000,ICAPEDESTRIANWALKINGSPEED,ICARIGHTTURNPEDESTRIANFACTOR,ICACONFIDENCELEVEL,ICASHAREOFCAVS\n
0,2327,NaN,NaN,0,0,0,0,0,1,-1.115252e+07,...,HCM 2010,HCM 2010,HCM 2010,HCM 2010,0,0,4km/h,Toronto,0.95,0.0
1,2328,NaN,NaN,0,0,0,0,0,1,-1.115252e+07,...,HCM 2010,HCM 2010,HCM 2010,HCM 2010,0,0,4km/h,Toronto,0.95,0.0
2,2329,NaN,NaN,0,0,0,0,0,1,-1.113956e+07,...,HCM 2010,HCM 2010,HCM 2010,HCM 2010,0,0,4km/h,Toronto,0.95,0.0
3,2330,NaN,NaN,0,0,0,0,0,1,-1.113952e+07,...,HCM 2010,HCM 2010,HCM 2010,HCM 2010,0,0,4km/h,Toronto,0.95,0.0
4,2331,NaN,NaN,0,0,0,0,0,1,-1.113499e+07,...,HCM 2010,HCM 2010,HCM 2010,HCM 2010,0,0,4km/h,Toronto,0.95,0.0


In [14]:
#display stop points
print(len(objDFs['Stoppoints']))
objDFs['Stoppoints'].head()

4842


,NO,STOPAREANO,CODE,NAME,TYPENO,TSYSSET,DIRECTED,NODENO,FROMNODENO,LINKNO,...,DEFDWELLTIME,DEPOTVEHCOMBSET,DEPOTCAP(1),DEPOTMINLAYOVERTIME(1),DEPOTVEHCOMBSTANDARD,DEPOTCAPSTANDARD,DEPOTMINLAYOVERTIMESTANDARD,COSTRATE1,COSTRATE2,COSTRATE3\n
0,2329,2329,2329,NaN,0,R,0,2329,NaN,NaN,...,0s,NaN,0,0s,0,0,0s,0.0,0.0,0.0
1,2330,2330,2330,NaN,0,R,0,2330,NaN,NaN,...,0s,NaN,0,0s,0,0,0s,0.0,0.0,0.0
2,2336,2336,2336,NaN,0,"Al,R",0,2336,NaN,NaN,...,0s,NaN,0,0s,0,0,0s,0.0,0.0,0.0
3,2354,2354,2354,NaN,0,R,0,2354,NaN,NaN,...,0s,NaN,0,0s,0,0,0s,0.0,0.0,0.0
4,2363,2363,2363,NaN,0,R,0,2363,NaN,NaN,...,0s,NaN,0,0s,0,0,0s,0.0,0.0,0.0


In [15]:
#get the different nodes in the cleaned network
n1 = list(objDFs['Links'] ['FROMNODENO'].unique())
n2 = list(objDFs['Links'] ['TONODENO'].unique())
#get the union
n = set(n1).union(set(n2))
#filter non stop nodes without links
objDFs['Nodes'] = objDFs['Nodes'].query('@n in NO | @nd in NO')
objDFs['Nodes']

,NO,CODE,NAME,TYPENO,CONTROLTYPE,MAINNODENO,USEMETHODIMPATNODE,METHODIMPATNODE,AUTOLINKORIENTATION,XCOORD,...,ICAMETHODALLWAYSTOP,ICAMETHODROUNDABOUT,ICAMETHODSIGNALIZED,ICAMETHODTWOWAYSTOP,VISTRODOANALYSIS,SIGNALWARRANTPOPISBELOW10000,ICAPEDESTRIANWALKINGSPEED,ICARIGHTTURNPEDESTRIANFACTOR,ICACONFIDENCELEVEL,ICASHAREOFCAVS\n
2,2329,NaN,NaN,0,0,0,0,0,1,-1.113956e+07,...,HCM 2010,HCM 2010,HCM 2010,HCM 2010,0,0,4km/h,Toronto,0.95,0.0
3,2330,NaN,NaN,0,0,0,0,0,1,-1.113952e+07,...,HCM 2010,HCM 2010,HCM 2010,HCM 2010,0,0,4km/h,Toronto,0.95,0.0
8,2335,NaN,NaN,0,0,0,0,0,1,-1.113530e+07,...,HCM 2010,HCM 2010,HCM 2010,HCM 2010,0,0,4km/h,Toronto,0.95,0.0
9,2336,NaN,NaN,0,0,0,0,0,1,-1.113493e+07,...,HCM 2010,HCM 2010,HCM 2010,HCM 2010,0,0,4km/h,Toronto,0.95,0.0
10,2337,NaN,NaN,0,0,0,0,0,1,-1.116986e+07,...,HCM 2010,HCM 2010,HCM 2010,HCM 2010,0,0,4km/h,Toronto,0.95,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128936,140810,NaN,NaN,0,0,0,0,0,1,-1.116174e+07,...,HCM 2010,HCM 2010,HCM 2010,HCM 2010,0,0,4km/h,Toronto,0.95,0.0
128937,140811,NaN,NaN,0,0,0,0,0,1,-1.115564e+07,...,HCM 2010,HCM 2010,HCM 2010,HCM 2010,0,0,4km/h,Toronto,0.95,0.0
128942,140816,NaN,NaN,0,0,0,0,0,1,-1.112689e+07,...,HCM 2010,HCM 2010,HCM 2010,HCM 2010,0,0,4km/h,Toronto,0.95,0.0
128943,140817,NaN,NaN,0,0,0,0,0,1,-1.113081e+07,...,HCM 2010,HCM 2010,HCM 2010,HCM 2010,0,0,4km/h,Toronto,0.95,0.0


* **From Turns clean nonstop nodes without links**

In [16]:
#display turns
print(len(objDFs['Turns']))
objDFs['Turns'].head()


805646


,FROMNODENO,VIANODENO,TONODENO,TYPENO,TSYSSET,CAPPRT,T0PRT,ADDVAL1,ADDVAL2,ADDVAL3,...,VISTROTURNONREDVOLUMECALCULATIONMETHOD,VISTROLRORDERNO,VISTROOTHERADJUSTFACTOR,VISTROLANEWIDTH,USEVISTROLANEWIDTH,VISTROOUTERCONTROL,VISTROTHRUCONTROL,VISTROINNERCONTROL,VISTROSGNO,VISTROOVLNO\n
0,107820,107948,107798,3,"Ca2_C,Ca1_C,Cu_C,Ca_BD,Cu_BD,Auto_C",99999,0s,0,0,0,...,ABSOLUTEVALUE,0,1.0,3.50m,0,PERMISSIVE,PERMISSIVE,PERMISSIVE,0,0
1,107820,107948,107819,2,"Ca2_C,Ca1_C,Cu_C,Ca_BD,Cu_BD,Auto_C",99999,0s,0,0,0,...,ABSOLUTEVALUE,0,1.0,3.50m,0,PERMISSIVE,PERMISSIVE,PERMISSIVE,0,0
3,107820,107948,137441,1,"Ca2_C,Ca1_C,Cu_C,Ca_BD,Cu_BD,Auto_C",99999,0s,0,0,0,...,ABSOLUTEVALUE,0,1.0,3.50m,0,PERMISSIVE,PERMISSIVE,PERMISSIVE,0,0
4,107948,107820,107813,2,"Ca2_C,Ca1_C,Cu_C,Ca_BD,Cu_BD,Auto_C,R",99999,0s,0,0,0,...,ABSOLUTEVALUE,0,1.0,3.50m,0,PERMISSIVE,PERMISSIVE,PERMISSIVE,0,0
6,107948,107820,108062,1,"Ca2_C,Ca1_C,Cu_C,Ca_BD,Cu_BD,Auto_C",99999,0s,0,0,0,...,ABSOLUTEVALUE,0,1.0,3.50m,0,PERMISSIVE,PERMISSIVE,PERMISSIVE,0,0


In [17]:
#filter non stop turns without links
objDFs['Turns'] = objDFs['Turns'].query('@n in (FROMNODENO | VIANODENO | TONODENO) | @nd in (FROMNODENO | VIANODENO | TONODENO)')
objDFs['Turns']

,FROMNODENO,VIANODENO,TONODENO,TYPENO,TSYSSET,CAPPRT,T0PRT,ADDVAL1,ADDVAL2,ADDVAL3,...,VISTROTURNONREDVOLUMECALCULATIONMETHOD,VISTROLRORDERNO,VISTROOTHERADJUSTFACTOR,VISTROLANEWIDTH,USEVISTROLANEWIDTH,VISTROOUTERCONTROL,VISTROTHRUCONTROL,VISTROINNERCONTROL,VISTROSGNO,VISTROOVLNO\n
1,107820,107948,107819,2,"Ca2_C,Ca1_C,Cu_C,Ca_BD,Cu_BD,Auto_C",99999,0s,0,0,0,...,ABSOLUTEVALUE,0,1.0,3.50m,0,PERMISSIVE,PERMISSIVE,PERMISSIVE,0,0
4,107948,107820,107813,2,"Ca2_C,Ca1_C,Cu_C,Ca_BD,Cu_BD,Auto_C,R",99999,0s,0,0,0,...,ABSOLUTEVALUE,0,1.0,3.50m,0,PERMISSIVE,PERMISSIVE,PERMISSIVE,0,0
17,2330,2329,38463,3,"Ca2_C,Ca1_C,Cu_C,Ca_BD,Cu_BD,Auto_C",99999,0s,0,0,0,...,ABSOLUTEVALUE,0,1.0,3.50m,0,PERMISSIVE,PERMISSIVE,PERMISSIVE,0,0
18,2330,2329,38543,1,"Ca2_C,Ca1_C,Cu_C,Ca_BD,Cu_BD,Auto_C,R",99999,0s,0,0,0,...,ABSOLUTEVALUE,0,1.0,3.50m,0,PERMISSIVE,PERMISSIVE,PERMISSIVE,0,0
22,2331,2332,122135,2,"Ca2_C,Ca1_C,Cu_C,Ca_BD,Cu_BD,Auto_C",99999,0s,0,0,0,...,ABSOLUTEVALUE,0,1.0,3.50m,0,PERMISSIVE,PERMISSIVE,PERMISSIVE,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1182055,71203,71219,70787,3,"Ca2_C,Ca1_C,Cu_C,Ca_BD,Cu_BD,Auto_C",99999,0s,0,0,0,...,ABSOLUTEVALUE,0,1.0,3.50m,0,PERMISSIVE,PERMISSIVE,PERMISSIVE,0,0
1182057,71203,71219,71203,4,"Ca2_C,Ca1_C,Cu_C,Ca_BD,Cu_BD,Auto_C",99999,0s,0,0,0,...,ABSOLUTEVALUE,0,1.0,3.50m,0,PERMISSIVE,PERMISSIVE,PERMISSIVE,0,0
1182060,71219,71203,71206,2,"Ca2_C,Ca1_C,Cu_C,Ca_BD,Cu_BD,Auto_C",99999,0s,0,0,0,...,ABSOLUTEVALUE,0,1.0,3.50m,0,PERMISSIVE,PERMISSIVE,PERMISSIVE,0,0
1182120,47351,47354,47557,3,"Ca2_C,Ca1_C,Cu_C,Ca_BD,Cu_BD,Auto_C",99999,0s,0,0,0,...,ABSOLUTEVALUE,0,1.0,3.50m,0,PERMISSIVE,PERMISSIVE,PERMISSIVE,0,0


* **clean non stop connectors without links**

In [18]:
#display turns
print(len(objDFs['Connectors']))
objDFs['Connectors'].head()


5187


,ZONENO,NODENO,DIRECTION,TYPENO,TSYSSET,LENGTH,T0_TSYS(AUTO_C),T0_TSYS(CA1_C),T0_TSYS(CA2_C),T0_TSYS(CA_BD),...,T0_TSYS(CU_C),T0_TSYS(W),WEIGHT(PRT),WEIGHT(PUT),ADDVAL1,ADDVAL2,ADDVAL3,LABELPOSRELX,LABELPOSRELY,EXPORTADO\n
0,1,15995,O,0,"Ca2_C,Ca1_C,Cu_C,Ca_BD,Cu_BD,Auto_C",0.200km,24s,0s,0s,0s,...,0s,120s,1,1,0,0,0,0.0,0.0,0
1,1,15995,D,0,"Ca2_C,Ca1_C,W,Cu_C,Ca_BD,Cu_BD,Auto_C",0.200km,24s,0s,0s,0s,...,0s,120s,1,1,0,0,0,0.0,0.0,0
2,1,17911,O,0,"Ca2_C,Ca1_C,Cu_C,Ca_BD,Cu_BD",0.200km,24s,0s,0s,0s,...,0s,120s,1,1,0,0,0,0.0,0.0,1
3,1,17911,D,0,"Ca2_C,Ca1_C,Cu_C,Ca_BD,Cu_BD,Auto_C",0.200km,24s,0s,0s,0s,...,0s,120s,1,1,0,0,0,0.0,0.0,1
4,1,17919,O,0,"Ca2_C,Ca1_C,Cu_C,Ca_BD,Cu_BD,Auto_C",0.200km,24s,0s,0s,0s,...,0s,120s,1,1,0,0,0,0.0,0.0,0


In [19]:
#filter non stop connectors without links
objDFs['Connectors'] = objDFs['Connectors'].query('@n in NODENO | @nd in NODENO')
objDFs['Connectors']

,ZONENO,NODENO,DIRECTION,TYPENO,TSYSSET,LENGTH,T0_TSYS(AUTO_C),T0_TSYS(CA1_C),T0_TSYS(CA2_C),T0_TSYS(CA_BD),...,T0_TSYS(CU_C),T0_TSYS(W),WEIGHT(PRT),WEIGHT(PUT),ADDVAL1,ADDVAL2,ADDVAL3,LABELPOSRELX,LABELPOSRELY,EXPORTADO\n
0,1,15995,O,0,"Ca2_C,Ca1_C,Cu_C,Ca_BD,Cu_BD,Auto_C",0.200km,24s,0s,0s,0s,...,0s,120s,1,1,0,0,0,0.0,0.0,0
1,1,15995,D,0,"Ca2_C,Ca1_C,W,Cu_C,Ca_BD,Cu_BD,Auto_C",0.200km,24s,0s,0s,0s,...,0s,120s,1,1,0,0,0,0.0,0.0,0
2,1,17911,O,0,"Ca2_C,Ca1_C,Cu_C,Ca_BD,Cu_BD",0.200km,24s,0s,0s,0s,...,0s,120s,1,1,0,0,0,0.0,0.0,1
3,1,17911,D,0,"Ca2_C,Ca1_C,Cu_C,Ca_BD,Cu_BD,Auto_C",0.200km,24s,0s,0s,0s,...,0s,120s,1,1,0,0,0,0.0,0.0,1
4,1,17919,O,0,"Ca2_C,Ca1_C,Cu_C,Ca_BD,Cu_BD,Auto_C",0.200km,24s,0s,0s,0s,...,0s,120s,1,1,0,0,0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5197,999,116337,D,0,"Ca2_C,Ca1_C,Cu_C,Ca_BD,Cu_BD,Auto_C",0.200km,24s,11116s,0s,0s,...,11116s,120s,4,1,0,0,0,0.0,0.0,1
5198,999,133489,O,0,W,0.200km,0s,0s,0s,0s,...,0s,120s,1,1,0,0,0,0.0,0.0,0
5199,999,133489,D,0,W,0.200km,0s,0s,0s,0s,...,0s,120s,1,1,0,0,0,0.0,0.0,0
5200,999,134362,O,0,"Ca2_C,Ca1_C,W,Cu_C,Ca_BD,Cu_BD,Auto_C",0.200km,24s,0s,0s,0s,...,0s,120s,10,1,0,0,0,0.0,0.0,0


In [20]:
#display zones
print(len(objDFs['Zones']))
objDFs['Zones'].head()

766


,NO,CODE,NAME,MAINZONENO,TYPENO,XCOORD,YCOORD,SURFACEID,RELATIVESTATE,SHAREPRTORIG,...,TXMOT_2030,TXMOT_2030_DOMS,TXMOT_2035,TXMOT_2035_DOMS,TXMOT_2040,TXMOT_2040_DOMS,TXMOT_2045,TXMOT_2045_DOMS,TXMOT_2050,TXMOT_2050_DOMS\n
0,1,0,NaN,13,0,-1.115399e+07,2.954884e+06,1,0,0,...,550.956,541.727,583.665,568.782,609.15,588.287,628.488,601.463,642.869,609.601
1,2,0,NaN,13,0,-1.115375e+07,2.956130e+06,2,0,0,...,550.956,541.727,583.665,568.782,609.15,588.287,628.488,601.463,642.869,609.601
2,3,0,NaN,13,0,-1.115570e+07,2.955264e+06,3,0,0,...,550.956,541.727,583.665,568.782,609.15,588.287,628.488,601.463,642.869,609.601
3,4,0,NaN,14,0,-1.115135e+07,2.955678e+06,4,0,0,...,550.956,541.727,583.665,568.782,609.15,588.287,628.488,601.463,642.869,609.601
4,5,0,NaN,14,0,-1.115217e+07,2.956862e+06,5,0,0,...,550.956,541.727,583.665,568.782,609.15,588.287,628.488,601.463,642.869,609.601


In [21]:
#validate that each zone has at least 1 connector
zones_c = objDFs['Connectors'].ZONENO.unique()
print(f"Zones: {len(objDFs['Zones'].NO)}")
if len(objDFs['Zones'].NO) == (sum(objDFs['Zones'].NO == zones_c)):
    print('Each zone has at least one connector')

Zones: 766
Each zone has at least one connector


* **Save dataframes as csv**

In [22]:
!mkdir output
for file in objDFs.keys():
    objDFs[file].to_csv('output/' + file + '.csv', index=False, float_format="%.3f")

* **Convert csv's to net file**

In [23]:
conv.csv2net('output/', 'output.net')

output.net


/home/alan/Documents/UToronto/Preparation-Network-for-GTAModel-Monterrey/Converters.py:145: DtypeWarning: Columns (242) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(tab_file)


* **compress output**

In [24]:
!rm -rf output
!rm output.zip
!zip output.zip output.net
!rm output.net

  adding: output.net (deflated 94%)


* **Remove Network folder with csvs**

In [25]:
!rm -rf Base_Network
!rm network.net